In [1]:
import random 

import numpy as np
import sympy as sp

from spatialmath import SO3, SE3
from roboticstoolbox import ET as et
from spatialmath.base import symbol

from manipylator import Robot
from manipylator.utils import render_robot_from_template, print_collapsible


# Introduction to Spatial Math
The `spatialmath` library provides primitives for manipulating [mathematical objects common in robotics](https://bdaiinstitute.github.io/spatialmath-python/intro.html#), such as `SE3`, `SO3` and `Twist`. Furthermore, this library overloads the `*` python operator to provide for the composition of these objects. Lastly, `spatialmath` is used to implement logic for Elementry Transform & Elementary Transform Sequence from the `robotictoolbox-python` library. For more details see [Robotics Toolbox Python documentation](https://petercorke.github.io/robotics-toolbox-python/intro.html#spatial-math-layer)

As a simple example, here's a 2 degrees-of-freedom planar manipulator, with a Z offset:

In [2]:
transform = et.Rx() * et.tx(1) * et.Ry() * et.tz(1) 
transform

Rx(q0) ⊕ tx(1) ⊕ Ry(q1) ⊕ tz(1)


In [3]:
type(transform)

roboticstoolbox.robot.ETS.ETS

This is a composition of elementary transforms (rotate around x,y,z and translate along xyz), also known as an ETS. This formulation supports symbolic computation directly, and lets us derive symbolic for forward kinematics:

In [4]:
phi, theta, psi = symbol('φ, ϴ, ψ')

type(transform.fkine([phi, theta]))

spatialmath.pose3d.SE3

In [5]:
transform.fkine([phi, theta])

  1.0*cos(ϴ)   0            1.0*sin(ϴ)   1.0*sin(ϴ) + 1.0  
  1.0*sin(φ)*sin(ϴ) 1.0*cos(φ)   -1.0*sin(φ)*cos(ϴ) -1.0*sin(φ)*cos(ϴ)  
  -1.0*sin(ϴ)*cos(φ) 1.0*sin(φ)   1.0*cos(φ)*cos(ϴ) 1.0*cos(φ)*cos(ϴ)  
  0            0            0            1.00000000000000  


Or equivalently, as [sympy.Matrix](https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html) objects:

In [6]:
symbolic_forward_kinematic = sp.Matrix(transform.fkine([phi, theta]).A)
symbolic_forward_kinematic

Matrix([
[        1.0*cos(ϴ),          0,         1.0*sin(ϴ),   1.0*sin(ϴ) + 1.0],
[ 1.0*sin(φ)*sin(ϴ), 1.0*cos(φ), -1.0*sin(φ)*cos(ϴ), -1.0*sin(φ)*cos(ϴ)],
[-1.0*sin(ϴ)*cos(φ), 1.0*sin(φ),  1.0*cos(φ)*cos(ϴ),  1.0*cos(φ)*cos(ϴ)],
[                 0,          0,                  0,                1.0]])

Which allows us to leverage sympy for our analysis, for example by calcualting the derivative and jacobian this homogenous transform:

In [7]:
sp.diff(symbolic_forward_kinematic, phi, theta)

Matrix([
[                0, 0,                 0,                 0],
[1.0*cos(φ)*cos(ϴ), 0, 1.0*sin(ϴ)*cos(φ), 1.0*sin(ϴ)*cos(φ)],
[1.0*sin(φ)*cos(ϴ), 0, 1.0*sin(φ)*sin(ϴ), 1.0*sin(φ)*sin(ϴ)],
[                0, 0,                 0,                 0]])

In [8]:
jacobian = symbolic_forward_kinematic[:3, 3].jacobian(sp.Matrix([phi, theta]))
jacobian

Matrix([
[                 0,         1.0*cos(ϴ)],
[-1.0*cos(φ)*cos(ϴ),  1.0*sin(φ)*sin(ϴ)],
[-1.0*sin(φ)*cos(ϴ), -1.0*sin(ϴ)*cos(φ)]])

Or numerically, if we prefer:

In [9]:
values = {phi: 0.1, theta: 0.6}
jacobian.subs(values).evalf(n=3)

Matrix([
[      0,  0.825],
[ -0.821, 0.0564],
[-0.0824, -0.562]])

In [10]:
values = {phi: 0, theta: 0}
jacobian.subs(values).evalf(n=3)

Matrix([
[   0, 1.0],
[-1.0,   0],
[   0,   0]])

# Symbolic Analysis of 6DOF Manipulator
The same powerful symbolic capabilities that spatialmath & SymPy provide are available for a generic, URDF specified robot.

In [11]:
with render_robot_from_template("robots/empiric") as robot_urdf:
    manny = Robot(robot_urdf)
print(manny.model)

ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                                                       │
│    1 │ carriage_1       │     0 │ base             │ SE3(-7.47e-06, -2.937e-06, 0.037; -180°, -7.914e-41°, -6.361e-15°) ⊕ Rz(q0) │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, -2.469e-06, -0.055; 180°, 90°, 180°) ⊕ Rz(q1)                   │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.355, -2.338e-06, -4.032e-06; -180°, -3.45e-15°, 3.069e-12°) ⊕ Rz(q2)  │
│ 

The ETS is available directly, and can be evaluated symbolically:

In [12]:

robot = manny.model

# Get the ETS
ets = robot.ets()

# Create symbolic variables for each joint
q_sym = sp.symbols(f'q0:{robot.n}')

# Substitute symbolic variables into the ETS to get a symbolic SE3
T = ets.eval(q_sym)  # This returns an SE3 object with symbolic entries


As expected, we get a 4x4 matrix, where each element is a symbolic statement. It should be clear here that it's impractical to calculate these kinds of statements by hand.

In [13]:
print("Shape:", T.shape)
print("First element:")
print_collapsible(T[0, 0])

Shape: (4, 4)
First element:


<details>

<summary>**Large Output** (click to expand - 189504 characters)</summary>

```python
1.95980826913726e-64*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.08215137033126e-27*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.08215137033126e-27*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.95980826913726e-64*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.96589028693246e-42*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 2.87158583120317e-33*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 2.87158583120317e-33*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.96589028693246e-42*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 6.51658569406023e-44*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 2.87158583121328e-33*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 2.87158583121328e-33*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) - 6.51658569406023e-44*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) + 3.12121673294836e-57*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*sin(q1) - 2.94606846035619e-22*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*cos(q1) + 2.94606846035619e-22*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*sin(q1) + 3.12121673294836e-57*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*cos(q1) - 1.0*(-5.22349862627469e-67*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.88426999996023e-30*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.88426999996023e-30*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 5.22349862627469e-67*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 5.23970909548155e-45*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) + 7.65366943324653e-36*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) + 7.65366943324653e-36*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 5.23970909548155e-45*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.73687278276e-46*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 7.65366943327348e-36*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 7.65366943327348e-36*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.73687278276e-46*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) + 1.0*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*sin(q1) + 1.0*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*cos(q1) + 1.05945152832833e-35*sin(q5) + 1.79962674409178e-49*cos(q5))*sin(q0) - 1.11021999999609e-16*(-5.22349862627469e-67*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.88426999996023e-30*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.88426999996023e-30*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 5.22349862627469e-67*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 5.23970909548155e-45*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) + 7.65366943324653e-36*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) + 7.65366943324653e-36*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 5.23970909548155e-45*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.73687278276e-46*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 7.65366943327348e-36*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 7.65366943327348e-36*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.73687278276e-46*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) + 1.0*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*sin(q1) + 1.0*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*cos(q1) + 1.05945152832833e-35*sin(q5) + 1.79962674409178e-49*cos(q5))*cos(q0) - 1.11021999999609e-16*(-1.81102969771882e-37*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 0.999999999979466*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 0.999999999979466*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 1.81102969771882e-37*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.81664999998135e-15*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) + 2.65358979332549e-6*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) + 2.65358979332549e-6*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.81664999998135e-15*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.02187999995938e-17*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 2.65358979333483e-6*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 2.65358979333483e-6*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 6.02187999995938e-17*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.88426999998054e-30*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*sin(q1) - 3.67320510334657e-6*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*cos(q1) + 3.67320510334657e-6*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*sin(q1) - 2.88426999998054e-30*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*cos(q1) + 3.67320510327115e-6*sin(q5) + 6.23945311666259e-20*cos(q5))*sin(q0) + 1.0*(-1.81102969771882e-37*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 0.999999999979466*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 0.999999999979466*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 1.81102969771882e-37*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.81664999998135e-15*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) + 2.65358979332549e-6*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) + 2.65358979332549e-6*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.81664999998135e-15*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.02187999995938e-17*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 2.65358979333483e-6*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 2.65358979333483e-6*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 6.02187999995938e-17*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.88426999998054e-30*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*sin(q1) - 3.67320510334657e-6*(1.08800600379579e-53*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 6.00766517050439e-17*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 6.00766517050439e-17*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 1.08800600379579e-53*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 1.09138249321089e-31*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 1.59418789784948e-22*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 1.59418789784948e-22*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 1.09138249321089e-31*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 1.0*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) + 5.35683001596074e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) + 5.35683001596074e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 1.0*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 2.20673863634942e-22*sin(q5) - 3.74845451727387e-36*cos(q5))*cos(q1) + 3.67320510334657e-6*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*sin(q1) - 2.88426999998054e-30*(4.80572992137468e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 2.65358979332549e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 2.65358979332549e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 4.80572992137468e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.82064389809424e-21*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 7.04153879138578e-12*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 7.04153879138578e-12*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 4.82064389809424e-21*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 5.35683e-14*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*sin(q2) - 0.999999999996479*(-4.8057299213916e-43*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 2.65358979333483e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) - 2.65358979333483e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.8057299213916e-43*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 5.9952e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 0.999999999996479*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 0.999999999996479*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) - 5.9952e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) + 9.74717957106587e-12*sin(q5) + 1.6556949106707e-25*cos(q5))*cos(q2) - 0.999999999996479*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*sin(q2) + 5.35683e-14*(3.28971898727442e-52*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 1.81649091196606e-15*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 1.81649091196606e-15*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) + 3.28971898727442e-52*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) - 1.0*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*sin(q3) - 5.99520048204328e-14*(1.0*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) + 1.0*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 4.93037999996674e-32*sin(q5) + 8.3749406832e-46*cos(q5))*cos(q3) - 5.99520048204328e-14*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*sin(q3) + 1.0*(-4.93037999996674e-32*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*sin(q4) - 3.67320510334657e-6*(-3.67320510332173e-6*sin(q5) - 3.67320510334657e-6*cos(q5))*cos(q4) + 3.67320510334657e-6*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*sin(q4) - 4.93037999996674e-32*(1.35094221312511e-11*sin(q5) - 0.999999999993254*cos(q5))*cos(q4) + 0.999999999986507*sin(q5) + 1.69863999998854e-14*cos(q5))*cos(q3) - 6.6723436880164e-21*sin(q5) - 1.13339098822886e-34*cos(q5))*cos(q2) - 9.74717957103155e-12*sin(q5) - 1.65569491066487e-25*cos(q5))*cos(q1) + 3.67320510327115e-6*sin(q5) + 6.23945311666259e-20*cos(q5))*cos(q0) - 3.97496393609425e-33*sin(q5) - 6.7520327404527e-47*cos(q5)
```

</details>

In [14]:
# For the curious what the entire matrix looks like, uncomment below
#print(sp.latex(T))  # LaTeX string

This approach, lets us convert our ETS model to a `sympy.Matrix` which provides a path for further analysis:

In [15]:
T_symbolic = sp.Matrix(T)

Like for example forward kinematics

In [16]:
subs_dict = {q: 0 for q in q_sym}
T_numeric = T_symbolic.subs(subs_dict).evalf(n=3)
T_numeric

Matrix([
[-8.98e-6, -3.67e-6,     1.0, -0.129],
[     1.0, -2.65e-6, 8.98e-6, 1.6e-6],
[ 2.65e-6,      1.0, 3.67e-6,  0.811],
[       0,        0,       0,    1.0]])

Or finding the analytic jacobian:

In [17]:
jacobian = T_symbolic[:3, 3].jacobian(sp.Matrix(q_sym))
print("Shape:", jacobian.shape)

Shape: (3, 6)


In [18]:
print("First element:")
print_collapsible(jacobian[0, 0])

First element:


<details>

<summary>**Large Output** (click to expand - 37557 characters)</summary>

```python
1.11021999999609e-16*(-5.23970909548155e-45*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) + 7.65366943324653e-36*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) + 7.65366943324653e-36*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 5.23970909548155e-45*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 7.65366943327348e-36*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.73687278276e-46*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.73687278276e-46*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 7.65366943327348e-36*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) + 1.0*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*sin(q1) + 1.0*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*cos(q1) + 6.68210367970787e-36*sin(q4) - 1.15370799998409e-31*cos(q4) - 2.46924e-6)*sin(q0) - 1.0*(-5.23970909548155e-45*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) + 7.65366943324653e-36*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) + 7.65366943324653e-36*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 5.23970909548155e-45*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 7.65366943327348e-36*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.73687278276e-46*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.73687278276e-46*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 7.65366943327348e-36*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) + 1.0*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*sin(q1) + 1.0*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*cos(q1) + 6.68210367970787e-36*sin(q4) - 1.15370799998409e-31*cos(q4) - 2.46924e-6)*cos(q0) - 1.0*(-1.81664999998135e-15*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) + 2.65358979332549e-6*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) + 2.65358979332549e-6*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.81664999998135e-15*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 2.65358979333483e-6*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 6.02187999995938e-17*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 6.02187999995938e-17*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 2.65358979333483e-6*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 2.88426999998054e-30*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*sin(q1) - 3.67320510334657e-6*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*cos(q1) + 3.67320510334657e-6*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*sin(q1) - 2.88426999998054e-30*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*cos(q1) + 2.31673999995243e-6*sin(q4) - 0.0399999999991786*cos(q4) - 0.0885038543428136)*sin(q0) - 1.11021999999609e-16*(-1.81664999998135e-15*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) + 2.65358979332549e-6*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) + 2.65358979332549e-6*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.81664999998135e-15*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 2.65358979333483e-6*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 6.02187999995938e-17*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 6.02187999995938e-17*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 2.65358979333483e-6*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 2.88426999998054e-30*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*sin(q1) - 3.67320510334657e-6*(1.09138249321089e-31*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 1.59418789784948e-22*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 1.59418789784948e-22*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 1.09138249321089e-31*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 5.35683001596074e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 1.0*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 1.0*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) + 5.35683001596074e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 1.39181982071143e-22*sin(q4) + 2.40306606820176e-18*cos(q4) + 0.355003)*cos(q1) + 3.67320510334657e-6*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*sin(q1) - 2.88426999998054e-30*(4.82064389809424e-21*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 7.04153879138578e-12*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 7.04153879138578e-12*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 4.82064389809424e-21*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 0.999999999996479*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*sin(q2) + 5.35683e-14*(-1.0*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 5.99520048204328e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 5.99520048204328e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) + 1.0*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) - 4.20833715538825e-21*sin(q4) + 7.26596364786423e-17*cos(q4) + 0.275499)*cos(q2) - 5.35683e-14*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*sin(q2) - 0.999999999996479*(5.9952e-14*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*sin(q3) - 0.999999999996479*(-0.04*sin(q4) - 2.31674e-6*cos(q4) + 1.31418e-7)*cos(q3) - 0.999999999996479*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*sin(q3) - 5.9952e-14*(-8.50986119112714e-12*sin(q4) + 1.46928204133863e-7*cos(q4) + 0.0885117999996728)*cos(q3) + 6.14767761781053e-12*sin(q4) - 1.06143591733393e-7*cos(q4) - 1.28747952397918e-7)*cos(q2) - 6.14767761778889e-12*sin(q4) + 1.06143591733019e-7*cos(q4) - 2.27556111258513e-6)*cos(q1) + 2.31673999995243e-6*sin(q4) - 0.0399999999991786*cos(q4) - 0.0885038543428136)*cos(q0)
```

</details>

As should be clear by now, the exact symbolic representation of 6DOF manipulator can be somewhat difficult to write down, but relatively straightforward to wrangle with SymPy. For example, we can expand all terms:

In [19]:
jacobian = jacobian.expand()
print_collapsible(jacobian[0, 0])

<details>

<summary>**Large Output** (click to expand - 8360 characters)</summary>

```python
1.46928204133346e-7*sin(q0)*sin(q1)*sin(q2)*sin(q3)*sin(q4) + 8.50986119111343e-12*sin(q0)*sin(q1)*sin(q2)*sin(q3)*cos(q4) - 4.82715478420789e-13*sin(q0)*sin(q1)*sin(q2)*sin(q3) - 3.5682666221358e-17*sin(q0)*sin(q1)*sin(q2)*sin(q4)*cos(q3) - 1.61394033179513e-26*sin(q0)*sin(q1)*sin(q2)*sin(q4) + 5.39697428991909e-13*sin(q0)*sin(q1)*sin(q2)*cos(q3)*cos(q4) + 3.25121995464045e-7*sin(q0)*sin(q1)*sin(q2)*cos(q3) + 2.78657135767523e-22*sin(q0)*sin(q1)*sin(q2)*cos(q4) + 1.01196433276332e-6*sin(q0)*sin(q1)*sin(q2) + 3.56826662219718e-17*sin(q0)*sin(q1)*sin(q3)*sin(q4)*cos(q2) + 4.88180852943946e-28*sin(q0)*sin(q1)*sin(q3)*sin(q4) - 5.39697428990009e-13*sin(q0)*sin(q1)*sin(q3)*cos(q2)*cos(q4) - 3.251219954629e-7*sin(q0)*sin(q1)*sin(q3)*cos(q2) + 3.80030042469844e-24*sin(q0)*sin(q1)*sin(q3)*cos(q4) + 2.28935914300088e-18*sin(q0)*sin(q1)*sin(q3) + 1.46928204132828e-7*sin(q0)*sin(q1)*sin(q4)*cos(q2)*cos(q3) + 2.25816807993916e-17*sin(q0)*sin(q1)*sin(q4)*cos(q2) - 1.03460064895725e-18*sin(q0)*sin(q1)*sin(q4)*cos(q3) + 2.25816807993916e-17*sin(q0)*sin(q1)*sin(q4) + 8.50986119108347e-12*sin(q0)*sin(q1)*cos(q2)*cos(q3)*cos(q4) - 4.82715478419088e-13*sin(q0)*sin(q1)*cos(q2)*cos(q3) - 3.89887182841262e-13*sin(q0)*sin(q1)*cos(q2)*cos(q4) - 4.7288710355102e-13*sin(q0)*sin(q1)*cos(q2) - 5.99225176840285e-23*sin(q0)*sin(q1)*cos(q3)*cos(q4) + 3.40069599948016e-24*sin(q0)*sin(q1)*cos(q3) - 3.89887182841262e-13*sin(q0)*sin(q1)*cos(q4) + 8.35864210486776e-12*sin(q0)*sin(q1) - 3.56826662220428e-17*sin(q0)*sin(q2)*sin(q3)*sin(q4)*cos(q1) + 1.06143591733393e-7*sin(q0)*sin(q2)*sin(q3)*sin(q4) + 5.39697428991909e-13*sin(q0)*sin(q2)*sin(q3)*cos(q1)*cos(q4) + 3.25121995464045e-7*sin(q0)*sin(q2)*sin(q3)*cos(q1) + 6.14767761780166e-12*sin(q0)*sin(q2)*sin(q3)*cos(q4) - 3.48734807616026e-13*sin(q0)*sin(q2)*sin(q3) - 1.46928204133346e-7*sin(q0)*sin(q2)*sin(q4)*cos(q1)*cos(q3) - 2.25816807994711e-17*sin(q0)*sin(q2)*sin(q4)*cos(q1) - 2.01665652783726e-17*sin(q0)*sin(q2)*sin(q4)*cos(q3) - 1.07969947535211e-26*sin(q0)*sin(q2)*sin(q4) - 8.50986119111343e-12*sin(q0)*sin(q2)*cos(q1)*cos(q3)*cos(q4) + 4.82715478420787e-13*sin(q0)*sin(q2)*cos(q1)*cos(q3) + 3.89887182842635e-13*sin(q0)*sin(q2)*cos(q1)*cos(q4) + 4.72887103552686e-13*sin(q0)*sin(q2)*cos(q1) + 3.89887182982515e-13*sin(q0)*sin(q2)*cos(q3)*cos(q4) + 2.34874009068825e-7*sin(q0)*sin(q2)*cos(q3) + 1.86417030025313e-22*sin(q0)*sin(q2)*cos(q4) + 7.31061334473952e-7*sin(q0)*sin(q2) + 1.46928204133863e-7*sin(q0)*sin(q3)*sin(q4)*cos(q1)*cos(q2) - 2.58074835410984e-38*sin(q0)*sin(q3)*sin(q4)*cos(q1) + 2.01665652787963e-17*sin(q0)*sin(q3)*sin(q4)*cos(q2) - 5.00843191998623e-17*sin(q0)*sin(q3)*sin(q4) + 8.50986119114339e-12*sin(q0)*sin(q3)*cos(q1)*cos(q2)*cos(q4) - 4.82715478422489e-13*sin(q0)*sin(q3)*cos(q1)*cos(q2) - 2.00901346437886e-34*sin(q0)*sin(q3)*cos(q1)*cos(q4) - 1.21026046077728e-28*sin(q0)*sin(q3)*cos(q1) - 3.89887182981142e-13*sin(q0)*sin(q3)*cos(q2)*cos(q4) - 2.34874009067998e-7*sin(q0)*sin(q3)*cos(q2) - 3.89887187049968e-13*sin(q0)*sin(q3)*cos(q4) - 2.34874009067998e-7*sin(q0)*sin(q3) - 3.5682666221429e-17*sin(q0)*sin(q4)*cos(q1)*cos(q2)*cos(q3) - 1.61394033179997e-26*sin(q0)*sin(q4)*cos(q1)*cos(q2) + 5.46937452760763e-29*sin(q0)*sin(q4)*cos(q1)*cos(q3) - 1.19377143131739e-27*sin(q0)*sin(q4)*cos(q1) + 1.06143591733019e-7*sin(q0)*sin(q4)*cos(q2)*cos(q3) + 1.6313414579335e-17*sin(q0)*sin(q4)*cos(q2) + 1.06143591733019e-7*sin(q0)*sin(q4)*cos(q3) - 2.31673999995243e-6*sin(q0)*sin(q4) + 5.3969742899381e-13*sin(q0)*cos(q1)*cos(q2)*cos(q3)*cos(q4) + 3.25121995465189e-7*sin(q0)*cos(q1)*cos(q2)*cos(q3) + 2.78657135768359e-22*sin(q0)*cos(q1)*cos(q2)*cos(q4) + 1.01196433276688e-6*sin(q0)*cos(q1)*cos(q2) + 3.16777968563489e-33*sin(q0)*cos(q1)*cos(q3)*cos(q4) - 1.79776419959143e-34*sin(q0)*cos(q1)*cos(q3) + 2.06112283867398e-23*sin(q0)*cos(q1)*cos(q4) + 1.30399883130334e-6*sin(q0)*cos(q1) + 6.14767761778001e-12*sin(q0)*cos(q2)*cos(q3)*cos(q4) - 3.48734807614797e-13*sin(q0)*cos(q2)*cos(q3) - 2.81661551651631e-13*sin(q0)*cos(q2)*cos(q4) - 3.41660842615055e-13*sin(q0)*cos(q2) + 6.14767761752197e-12*sin(q0)*cos(q3)*cos(q4) - 3.48890258420716e-13*sin(q0)*cos(q3) + 0.0399999999991786*sin(q0)*cos(q4) + 0.0885038543428136*sin(q0) + 8.50532037910711e-12*sin(q1)*sin(q2)*sin(q3)*sin(q4)*cos(q0) - 1.46928204133609e-7*sin(q1)*sin(q2)*sin(q3)*cos(q0)*cos(q4) - 0.0885117999993612*sin(q1)*sin(q2)*sin(q3)*cos(q0) + 0.0399999999998592*sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3) + 6.14767761781053e-12*sin(q1)*sin(q2)*sin(q4)*cos(q0) + 2.31673999999183e-6*sin(q1)*sin(q2)*cos(q0)*cos(q3)*cos(q4) - 1.31418010047423e-7*sin(q1)*sin(q2)*cos(q0)*cos(q3) - 1.06143591733393e-7*sin(q1)*sin(q2)*cos(q0)*cos(q4) - 1.28747967155931e-7*sin(q1)*sin(q2)*cos(q0) - 0.04*sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2) + 3.00889820057036e-33*sin(q1)*sin(q3)*sin(q4)*cos(q0) - 2.31673999999998e-6*sin(q1)*sin(q3)*cos(q0)*cos(q2)*cos(q4) + 1.31418010047886e-7*sin(q1)*sin(q3)*cos(q0)*cos(q2) + 2.34231167415631e-29*sin(q1)*sin(q3)*cos(q0)*cos(q4) + 1.41104440378894e-23*sin(q1)*sin(q3)*cos(q0) + 8.50532037894426e-12*sin(q1)*sin(q4)*cos(q0)*cos(q2)*cos(q3) + 4.20800783475084e-21*sin(q1)*sin(q4)*cos(q0)*cos(q2) - 6.37675159151279e-24*sin(q1)*sin(q4)*cos(q0)*cos(q3) + 1.39181982073651e-22*sin(q1)*sin(q4)*cos(q0) - 1.46928204134126e-7*sin(q1)*cos(q0)*cos(q2)*cos(q3)*cos(q4) - 0.0885117999996728*sin(q1)*cos(q0)*cos(q2)*cos(q3) - 7.26539505469036e-17*sin(q1)*cos(q0)*cos(q2)*cos(q4) - 0.275499*sin(q1)*cos(q0)*cos(q2) - 3.69331887036998e-28*sin(q1)*cos(q0)*cos(q3)*cos(q4) + 2.09601585392321e-29*sin(q1)*cos(q0)*cos(q3) - 2.40306606824504e-18*sin(q1)*cos(q0)*cos(q4) - 0.355003*sin(q1)*cos(q0) + 0.0399999999998592*sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1) + 1.17842738413836e-23*sin(q2)*sin(q3)*sin(q4)*cos(q0) + 2.31673999999183e-6*sin(q2)*sin(q3)*cos(q0)*cos(q1)*cos(q4) - 1.31418010047424e-7*sin(q2)*sin(q3)*cos(q0)*cos(q1) + 6.8252746448119e-28*sin(q2)*sin(q3)*cos(q0)*cos(q4) - 3.87172358110111e-29*sin(q2)*sin(q3)*cos(q0) - 8.50532037892069e-12*sin(q2)*sin(q4)*cos(q0)*cos(q1)*cos(q3) - 4.208007834737e-21*sin(q2)*sin(q4)*cos(q0)*cos(q1) - 2.23893241032766e-33*sin(q2)*sin(q4)*cos(q0)*cos(q3) - 1.19870395152123e-42*sin(q2)*sin(q4)*cos(q0) + 1.46928204133609e-7*sin(q2)*cos(q0)*cos(q1)*cos(q3)*cos(q4) + 0.0885117999993612*sin(q2)*cos(q0)*cos(q1)*cos(q3) + 7.26539505466648e-17*sin(q2)*cos(q0)*cos(q1)*cos(q4) + 0.27549899999903*sin(q2)*cos(q0)*cos(q1) + 4.32860548289335e-29*sin(q2)*cos(q0)*cos(q3)*cos(q4) + 2.6076182234748e-23*sin(q2)*cos(q0)*cos(q3) + 2.06963915073979e-38*sin(q2)*cos(q0)*cos(q4) + 8.11638914756834e-23*sin(q2)*cos(q0) + 8.50532037908354e-12*sin(q3)*sin(q4)*cos(q0)*cos(q1)*cos(q2) + 1.32903238237139e-22*sin(q3)*sin(q4)*cos(q0)*cos(q1) + 2.2389324103747e-33*sin(q3)*sin(q4)*cos(q0)*cos(q2) - 5.56046128618768e-33*sin(q3)*sin(q4)*cos(q0) - 1.46928204133091e-7*sin(q3)*cos(q0)*cos(q1)*cos(q2)*cos(q4) - 0.0885117999990496*sin(q3)*cos(q0)*cos(q1)*cos(q2) + 1.03460066012542e-18*sin(q3)*cos(q0)*cos(q1)*cos(q4) + 6.23259273193076e-13*sin(q3)*cos(q0)*cos(q1) - 4.32860548287811e-29*sin(q3)*cos(q0)*cos(q2)*cos(q4) - 2.60761822346562e-23*sin(q3)*cos(q0)*cos(q2) - 4.32860552805103e-29*sin(q3)*cos(q0)*cos(q4) - 2.60761822346562e-23*sin(q3)*cos(q0) + 0.0399999999997183*sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3) + 6.14767761778889e-12*sin(q4)*cos(q0)*cos(q1)*cos(q2) - 2.81661551655431e-13*sin(q4)*cos(q0)*cos(q1)*cos(q3) + 6.14767761778889e-12*sin(q4)*cos(q0)*cos(q1) + 1.17842738413421e-23*sin(q4)*cos(q0)*cos(q2)*cos(q3) + 1.8111479134206e-33*sin(q4)*cos(q0)*cos(q2) + 1.17842738413421e-23*sin(q4)*cos(q0)*cos(q3) - 2.5720910827382e-22*sin(q4)*cos(q0) + 2.31673999998367e-6*cos(q0)*cos(q1)*cos(q2)*cos(q3)*cos(q4) - 1.31418010046961e-7*cos(q0)*cos(q1)*cos(q2)*cos(q3) - 1.06143591733019e-7*cos(q0)*cos(q1)*cos(q2)*cos(q4) - 1.28747967155478e-7*cos(q0)*cos(q1)*cos(q2) - 1.63134145788468e-17*cos(q0)*cos(q1)*cos(q3)*cos(q4) + 9.25811628754915e-19*cos(q0)*cos(q1)*cos(q3) - 1.06143591733019e-7*cos(q0)*cos(q1)*cos(q4) + 2.27556111258513e-6*cos(q0)*cos(q1) + 6.82527464478787e-28*cos(q0)*cos(q2)*cos(q3)*cos(q4) - 3.87172358108747e-29*cos(q0)*cos(q2)*cos(q3) - 3.12706287873581e-29*cos(q0)*cos(q2)*cos(q4) - 3.7931870068676e-29*cos(q0)*cos(q2) + 6.82527464450139e-28*cos(q0)*cos(q3)*cos(q4) - 3.87344942702494e-29*cos(q0)*cos(q3) + 4.44087999989329e-18*cos(q0)*cos(q4) + 2.46924000000983e-6*cos(q0)
```

</details>

Similarly, studying the coefficients of the above expression might be interesting. Specifically, we might be able to find opportunities for simplification:

In [20]:
coeff_dict = jacobian[0, 0].as_coefficients_dict()
print_collapsible(coeff_dict)

<details>

<summary>**Large Output** (click to expand - 8514 characters)</summary>

```python
defaultdict(<class 'int'>, {cos(q0): 2.46924000000983e-6, sin(q0): 0.0885038543428136, sin(q2)*cos(q0): 8.11638914756834e-23, sin(q0)*sin(q2): 7.31061334473952e-7, sin(q0)*sin(q1): 8.35864210486776e-12, cos(q0)*cos(q1): 2.27556111258513e-6, cos(q0)*cos(q4): 4.44087999989329e-18, sin(q0)*cos(q4): 0.0399999999991786, sin(q0)*cos(q1): 1.30399883130334e-6, sin(q0)*sin(q3): -2.34874009067998e-7, sin(q3)*cos(q0): -2.60761822346562e-23, sin(q4)*cos(q0): -2.57209108273820e-22, sin(q0)*sin(q4): -2.31673999995243e-6, sin(q1)*cos(q0): -0.355003000000000, cos(q0)*cos(q2): -3.79318700686760e-29, sin(q0)*cos(q2): -3.41660842615055e-13, cos(q0)*cos(q3): -3.87344942702494e-29, sin(q0)*cos(q3): -3.48890258420716e-13, sin(q1)*sin(q3)*cos(q0): 1.41104440378894e-23, sin(q0)*sin(q2)*cos(q1): 4.72887103552686e-13, cos(q0)*cos(q1)*cos(q3): 9.25811628754915e-19, sin(q1)*cos(q0)*cos(q3): 2.09601585392321e-29, sin(q0)*cos(q1)*cos(q2): 1.01196433276688e-6, sin(q2)*cos(q0)*cos(q1): 0.275498999999030, sin(q4)*cos(q0)*cos(q2): 1.81114791342060e-33, sin(q0)*sin(q1)*sin(q3): 2.28935914300088e-18, sin(q0)*cos(q3)*cos(q4): 6.14767761752197e-12, sin(q4)*cos(q0)*cos(q1): 6.14767761778889e-12, sin(q2)*cos(q0)*cos(q4): 2.06963915073979e-38, sin(q2)*cos(q0)*cos(q3): 2.60761822347480e-23, sin(q0)*sin(q2)*cos(q3): 2.34874009068825e-7, sin(q0)*sin(q1)*cos(q3): 3.40069599948016e-24, sin(q0)*sin(q1)*sin(q2): 1.01196433276332e-6, sin(q0)*sin(q4)*cos(q2): 1.63134145793350e-17, sin(q1)*sin(q4)*cos(q0): 1.39181982073651e-22, sin(q3)*cos(q0)*cos(q1): 6.23259273193076e-13, sin(q0)*cos(q1)*cos(q4): 2.06112283867398e-23, sin(q0)*sin(q1)*sin(q4): 2.25816807993916e-17, cos(q0)*cos(q3)*cos(q4): 6.82527464450139e-28, sin(q0)*sin(q2)*cos(q4): 1.86417030025313e-22, sin(q4)*cos(q0)*cos(q3): 1.17842738413421e-23, sin(q0)*sin(q4)*cos(q3): 1.06143591733019e-7, sin(q0)*sin(q1)*cos(q2): -4.72887103551020e-13, sin(q0)*sin(q3)*cos(q1): -1.21026046077728e-28, cos(q0)*cos(q2)*cos(q4): -3.12706287873581e-29, sin(q0)*cos(q2)*cos(q4): -2.81661551651631e-13, sin(q1)*cos(q0)*cos(q4): -2.40306606824504e-18, sin(q0)*cos(q2)*cos(q3): -3.48734807614797e-13, sin(q0)*sin(q1)*cos(q4): -3.89887182841262e-13, cos(q0)*cos(q2)*cos(q3): -3.87172358108747e-29, sin(q3)*cos(q0)*cos(q4): -4.32860552805103e-29, sin(q0)*sin(q3)*cos(q4): -3.89887187049968e-13, sin(q0)*sin(q3)*sin(q4): -5.00843191998623e-17, cos(q0)*cos(q1)*cos(q2): -1.28747967155478e-7, sin(q1)*sin(q2)*cos(q0): -1.28747967155931e-7, sin(q1)*cos(q0)*cos(q2): -0.275499000000000, sin(q0)*sin(q4)*cos(q1): -1.19377143131739e-27, sin(q2)*sin(q3)*cos(q0): -3.87172358110111e-29, sin(q0)*sin(q2)*sin(q3): -3.48734807616026e-13, sin(q2)*sin(q4)*cos(q0): -1.19870395152123e-42, sin(q0)*sin(q2)*sin(q4): -1.07969947535211e-26, cos(q0)*cos(q1)*cos(q4): -1.06143591733019e-7, sin(q3)*sin(q4)*cos(q0): -5.56046128618768e-33, sin(q0)*cos(q1)*cos(q3): -1.79776419959143e-34, sin(q3)*cos(q0)*cos(q2): -2.60761822346562e-23, sin(q0)*sin(q3)*cos(q2): -2.34874009067998e-7, sin(q1)*sin(q3)*cos(q0)*cos(q4): 2.34231167415631e-29, sin(q1)*sin(q4)*cos(q0)*cos(q2): 4.20800783475084e-21, sin(q0)*sin(q2)*sin(q3)*cos(q1): 3.25121995464045e-7, sin(q0)*sin(q1)*sin(q2)*cos(q3): 3.25121995464045e-7, sin(q0)*sin(q2)*cos(q3)*cos(q4): 3.89887182982515e-13, sin(q0)*sin(q4)*cos(q1)*cos(q3): 5.46937452760763e-29, sin(q0)*sin(q1)*sin(q3)*sin(q4): 4.88180852943946e-28, sin(q3)*sin(q4)*cos(q0)*cos(q1): 1.32903238237139e-22, sin(q2)*cos(q0)*cos(q1)*cos(q4): 7.26539505466648e-17, sin(q1)*sin(q2)*sin(q4)*cos(q0): 6.14767761781053e-12, sin(q2)*cos(q0)*cos(q3)*cos(q4): 4.32860548289335e-29, sin(q0)*sin(q2)*cos(q1)*cos(q3): 4.82715478420787e-13, sin(q0)*cos(q1)*cos(q2)*cos(q3): 3.25121995465189e-7, sin(q0)*sin(q3)*sin(q4)*cos(q2): 2.01665652787963e-17, sin(q4)*cos(q0)*cos(q1)*cos(q2): 6.14767761778889e-12, sin(q0)*cos(q1)*cos(q3)*cos(q4): 3.16777968563489e-33, sin(q1)*sin(q3)*cos(q0)*cos(q2): 1.31418010047886e-7, sin(q0)*sin(q1)*sin(q3)*cos(q4): 3.80030042469844e-24, sin(q3)*cos(q0)*cos(q1)*cos(q4): 1.03460066012542e-18, sin(q0)*sin(q1)*sin(q2)*cos(q4): 2.78657135767523e-22, sin(q0)*cos(q1)*cos(q2)*cos(q4): 2.78657135768359e-22, sin(q2)*cos(q0)*cos(q1)*cos(q3): 0.0885117999993612, sin(q3)*sin(q4)*cos(q0)*cos(q2): 2.23893241037470e-33, sin(q0)*sin(q1)*sin(q4)*cos(q2): 2.25816807993916e-17, cos(q0)*cos(q2)*cos(q3)*cos(q4): 6.82527464478787e-28, sin(q2)*sin(q3)*cos(q0)*cos(q4): 6.82527464481190e-28, sin(q0)*cos(q2)*cos(q3)*cos(q4): 6.14767761778001e-12, sin(q0)*sin(q2)*sin(q3)*cos(q4): 6.14767761780166e-12, sin(q0)*sin(q2)*cos(q1)*cos(q4): 3.89887182842635e-13, sin(q4)*cos(q0)*cos(q2)*cos(q3): 1.17842738413421e-23, sin(q2)*sin(q3)*sin(q4)*cos(q0): 1.17842738413836e-23, sin(q0)*sin(q4)*cos(q2)*cos(q3): 1.06143591733019e-7, sin(q0)*sin(q2)*sin(q3)*sin(q4): 1.06143591733393e-7, sin(q1)*sin(q3)*sin(q4)*cos(q0): 3.00889820057036e-33, sin(q1)*cos(q0)*cos(q2)*cos(q4): -7.26539505469036e-17, sin(q3)*cos(q0)*cos(q1)*cos(q2): -0.0885117999990496, sin(q2)*sin(q4)*cos(q0)*cos(q3): -2.23893241032766e-33, sin(q1)*sin(q4)*cos(q0)*cos(q3): -6.37675159151279e-24, cos(q0)*cos(q1)*cos(q3)*cos(q4): -1.63134145788468e-17, sin(q0)*sin(q1)*sin(q2)*sin(q4): -1.61394033179513e-26, sin(q0)*sin(q1)*cos(q2)*cos(q4): -3.89887182841262e-13, sin(q0)*sin(q1)*sin(q2)*sin(q3): -4.82715478420789e-13, sin(q0)*sin(q1)*sin(q4)*cos(q3): -1.03460064895725e-18, sin(q4)*cos(q0)*cos(q1)*cos(q3): -2.81661551655431e-13, sin(q2)*sin(q4)*cos(q0)*cos(q1): -4.20800783473700e-21, sin(q0)*sin(q1)*sin(q3)*cos(q2): -3.25121995462900e-7, sin(q1)*cos(q0)*cos(q2)*cos(q3): -0.0885117999996728, sin(q0)*sin(q2)*sin(q4)*cos(q1): -2.25816807994711e-17, sin(q0)*sin(q3)*cos(q1)*cos(q4): -2.00901346437886e-34, sin(q0)*sin(q1)*cos(q2)*cos(q3): -4.82715478419088e-13, sin(q0)*sin(q3)*cos(q1)*cos(q2): -4.82715478422489e-13, sin(q0)*sin(q3)*sin(q4)*cos(q1): -2.58074835410984e-38, cos(q0)*cos(q1)*cos(q2)*cos(q3): -1.31418010046961e-7, sin(q1)*sin(q2)*cos(q0)*cos(q3): -1.31418010047423e-7, sin(q2)*sin(q3)*cos(q0)*cos(q1): -1.31418010047424e-7, sin(q0)*sin(q1)*cos(q3)*cos(q4): -5.99225176840285e-23, sin(q0)*sin(q4)*cos(q1)*cos(q2): -1.61394033179997e-26, sin(q1)*sin(q2)*sin(q3)*cos(q0): -0.0885117999993612, sin(q0)*sin(q2)*sin(q4)*cos(q3): -2.01665652783726e-17, sin(q3)*cos(q0)*cos(q2)*cos(q4): -4.32860548287811e-29, sin(q0)*sin(q3)*cos(q2)*cos(q4): -3.89887182981142e-13, cos(q0)*cos(q1)*cos(q2)*cos(q4): -1.06143591733019e-7, sin(q1)*sin(q2)*cos(q0)*cos(q4): -1.06143591733393e-7, sin(q1)*cos(q0)*cos(q3)*cos(q4): -3.69331887036998e-28, sin(q0)*sin(q3)*cos(q1)*cos(q2)*cos(q4): 8.50986119114339e-12, sin(q2)*sin(q3)*cos(q0)*cos(q1)*cos(q4): 2.31673999999183e-6, sin(q1)*sin(q2)*cos(q0)*cos(q3)*cos(q4): 2.31673999999183e-6, sin(q0)*sin(q1)*sin(q2)*sin(q3)*sin(q4): 1.46928204133346e-7, sin(q0)*sin(q3)*sin(q4)*cos(q1)*cos(q2): 1.46928204133863e-7, sin(q1)*sin(q4)*cos(q0)*cos(q2)*cos(q3): 8.50532037894426e-12, sin(q3)*sin(q4)*cos(q0)*cos(q1)*cos(q2): 8.50532037908354e-12, sin(q1)*sin(q2)*sin(q3)*sin(q4)*cos(q0): 8.50532037910711e-12, sin(q0)*sin(q1)*cos(q2)*cos(q3)*cos(q4): 8.50986119108347e-12, sin(q0)*sin(q1)*sin(q2)*sin(q3)*cos(q4): 8.50986119111343e-12, sin(q0)*sin(q2)*sin(q3)*cos(q1)*cos(q4): 5.39697428991909e-13, sin(q0)*sin(q1)*sin(q2)*cos(q3)*cos(q4): 5.39697428991909e-13, sin(q0)*cos(q1)*cos(q2)*cos(q3)*cos(q4): 5.39697428993810e-13, cos(q0)*cos(q1)*cos(q2)*cos(q3)*cos(q4): 2.31673999998367e-6, sin(q0)*sin(q1)*sin(q4)*cos(q2)*cos(q3): 1.46928204132828e-7, sin(q2)*cos(q0)*cos(q1)*cos(q3)*cos(q4): 1.46928204133609e-7, sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3): 0.0399999999997183, sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1): 0.0399999999998592, sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3): 0.0399999999998592, sin(q0)*sin(q1)*sin(q3)*sin(q4)*cos(q2): 3.56826662219718e-17, sin(q3)*cos(q0)*cos(q1)*cos(q2)*cos(q4): -1.46928204133091e-7, sin(q0)*sin(q2)*sin(q4)*cos(q1)*cos(q3): -1.46928204133346e-7, sin(q1)*cos(q0)*cos(q2)*cos(q3)*cos(q4): -1.46928204134126e-7, sin(q2)*sin(q4)*cos(q0)*cos(q1)*cos(q3): -8.50532037892069e-12, sin(q0)*sin(q2)*cos(q1)*cos(q3)*cos(q4): -8.50986119111343e-12, sin(q0)*sin(q1)*sin(q3)*cos(q2)*cos(q4): -5.39697428990009e-13, sin(q1)*sin(q3)*cos(q0)*cos(q2)*cos(q4): -2.31673999999998e-6, sin(q1)*sin(q2)*sin(q3)*cos(q0)*cos(q4): -1.46928204133609e-7, sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2): -0.0400000000000000, sin(q0)*sin(q1)*sin(q2)*sin(q4)*cos(q3): -3.56826662213580e-17, sin(q0)*sin(q4)*cos(q1)*cos(q2)*cos(q3): -3.56826662214290e-17, sin(q0)*sin(q2)*sin(q3)*sin(q4)*cos(q1): -3.56826662220428e-17})
```

</details>

The above (likely over 100) coefficients should be understood as a just another way of writing down the expression. For practical reasons, we might even want to reconstruct another expression from it's elements:

In [21]:
threshold = 1e-5 # Chosen arbitrarily to make a point
# Filter coeff_dict
significant_terms = {term: coeff for term, coeff in coeff_dict.items() 
                    if abs(coeff) > threshold} 

# Recreate expression from significant terms
filtered_expression = sum(coeff * term for term, coeff in significant_terms.items())
print("Filtered expression:")
print_collapsible(filtered_expression)

Filtered expression:


<details>

<summary>**Large Output** (click to expand - 610 characters)</summary>

```python
0.0399999999991786*sin(q0)*cos(q4) + 0.0885038543428136*sin(q0) - 0.0885117999993612*sin(q1)*sin(q2)*sin(q3)*cos(q0) + 0.0399999999998592*sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3) - 0.04*sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2) - 0.0885117999996728*sin(q1)*cos(q0)*cos(q2)*cos(q3) - 0.275499*sin(q1)*cos(q0)*cos(q2) - 0.355003*sin(q1)*cos(q0) + 0.0399999999998592*sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1) + 0.0885117999993612*sin(q2)*cos(q0)*cos(q1)*cos(q3) + 0.27549899999903*sin(q2)*cos(q0)*cos(q1) - 0.0885117999990496*sin(q3)*cos(q0)*cos(q1)*cos(q2) + 0.0399999999997183*sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3)
```

</details>

In [22]:
filtered_expression.as_coefficients_dict()

defaultdict(int,
            {sin(q0): 0.0885038543428136,
             sin(q0)*cos(q4): 0.0399999999991786,
             sin(q1)*cos(q0): -0.355003000000000,
             sin(q2)*cos(q0)*cos(q1): 0.275498999999030,
             sin(q1)*cos(q0)*cos(q2): -0.275499000000000,
             sin(q2)*cos(q0)*cos(q1)*cos(q3): 0.0885117999993612,
             sin(q3)*cos(q0)*cos(q1)*cos(q2): -0.0885117999990496,
             sin(q1)*cos(q0)*cos(q2)*cos(q3): -0.0885117999996728,
             sin(q1)*sin(q2)*sin(q3)*cos(q0): -0.0885117999993612,
             sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3): 0.0399999999997183,
             sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1): 0.0399999999998592,
             sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3): 0.0399999999998592,
             sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2): -0.0400000000000000})

The above is to say that even when working symbolicly, an order of magnitude analysis might be useful to make a problem tractable. Calculating a symbolic determinant might be impractical on the original expression, but practically trivial in the simplified case.

Due note that the above example is only instructional, and in a practical case use `sympy.nsimplify`:

In [23]:
filtered_expression = sp.nsimplify(jacobian[0, 0], tolerance=1e-5)
filtered_expression.as_coefficients_dict()


defaultdict(int,
            {sin(q0): 2365/26722,
             sin(q1)*cos(q0): -25442/71667,
             sin(q0)*cos(q4): 1/25,
             sin(q1)*cos(q0)*cos(q2): -14051/51002,
             sin(q2)*cos(q0)*cos(q1): 14051/51002,
             sin(q3)*cos(q0)*cos(q1)*cos(q2): -8690/98179,
             sin(q1)*cos(q0)*cos(q2)*cos(q3): -8690/98179,
             sin(q1)*sin(q2)*sin(q3)*cos(q0): -8690/98179,
             sin(q2)*cos(q0)*cos(q1)*cos(q3): 8690/98179,
             sin(q1)*sin(q3)*sin(q4)*cos(q0)*cos(q2): -1/25,
             sin(q4)*cos(q0)*cos(q1)*cos(q2)*cos(q3): 1/25,
             sin(q2)*sin(q3)*sin(q4)*cos(q0)*cos(q1): 1/25,
             sin(q1)*sin(q2)*sin(q4)*cos(q0)*cos(q3): 1/25})

Or for the entire jacobian:

In [ ]:
# We want elementwise filtering, therefore use the .applyfunc method
# sympy.nsimplify(sympy.Matrix) does not work as we'd like in this case
filtered_jacobian = jacobian.applyfunc(lambda x: sp.nsimplify(x, tolerance=1e-5))

The simplification can even be quantified by counting the number of terms in the expression:

In [ ]:
def count_total_terms(matrix):
    return sum(
        len(element.args) if element.is_Add else 1
        for element in matrix
    )

print(f"Total terms: {count_total_terms(jacobian)}")
print(f"Total filtered terms: {count_total_terms(filtered_jacobian)}")

# Calculating Manipulability
By taking the oppurtunity to simplify our Jacobian, further symbolic & numeric calculations can be sped up significantly without loosing much precision. We can now try to calculate a [manipulability metric](https://petercorke.github.io/robotics-toolbox-python/arm_superclass.html#roboticstoolbox.robot.Robot.Robot.manipulability):

In [ ]:
# manipulability = sympy.sqrt((jacobian * jacobian.T).det())  # J*J^T is 3×3
# manipulability = sympy.sqrt((filtered_jacobian * filtered_jacobian.T).det()) 


Unfortunately, the above expression does not return in a timely manner, even for the filtered expression. Specifically, the calculating the determinant is the root cause, which could potentially be optimized around by filtering more agressively or waiting longer. Alternatively, a numeric approach might prove more efficient in this case.

Specifically, we can derive a numeric function from our expression and use numpy to efficiently calculate the determinant numerically.

In [ ]:
M = filtered_jacobian * filtered_jacobian.T
M_fn = sp.lambdify(q_sym, M, modules='numpy') # Make a wrapper function that handles numpy object

In [ ]:
# Create a NxNxN meshgrid for efficiently calculating M_fn
N = 100

q1_vals = np.linspace(-np.pi/2, np.pi/2, N)
q2_vals = np.linspace(-np.pi/2, np.pi/2, N)
q3_vals = np.linspace(-np.pi/2, np.pi/2, N)
Q1, Q2, Q3 = np.meshgrid(q1_vals, q2_vals, q3_vals, indexing='ij')

# Calculate M_fn at meshgrid
# We're simplifying and only looking at DOFs that are likely to have singularities
args = [0, Q1, Q2, Q3, 0, 0]   # [q0, q1, q2, q3, q4, q5]

M_grid = np.asarray(M_fn(*args)) # evaluate; shape is (3,3,N,N,N)
M_grid = np.moveaxis(M_grid, (0,1), (-2,-1))  # -> (N,N,N,3,3) so we can pass to np.linalg.det

In [ ]:
# manipulability on the grid: sqrt(det(J*J^T))
det_grid = np.linalg.det(M_grid)
manip_grid = np.sqrt(det_grid)

Once calculated, we can explore the result interactively (if run locally using the lab environment). In this interpretation, low manipulability values indicate a near-singularity configuratation:

In [ ]:
import xarray as xr, holoviews as hv, hvplot.xarray, panel as pn
hv.extension('bokeh')

da = xr.DataArray(manip_grid, 
                  coords={'q1': q1_vals, 'q2': q2_vals, 'q3': q3_vals},
                  dims=('q1','q2','q3'), 
                  name='manip')

idx = pn.widgets.IntSlider(name='q3', start=0, end=len(q3_vals)-1, value=len(q3_vals)//2)

@pn.depends(idx)
def view(k):
    return da.isel(q3=k).hvplot.image(x='q1', y='q2', cmap='Viridis', colorbar=True, aspect=1)

pn.Column(idx, "# Yoshikawa Manipulability", view).servable()

Note the appearance of a low manipulability when q2, q3 are close to zero, indicated by the horizontal darker area. If q3 moves away from 0, this low manipulability area can disappear: 

In [ ]:
pn.Column("# Yoshikawa Manipulability for 2 Values of q3" ,pn.Row(view(0), view(50)))

Note: this manipulability measure has units and is scale-dependent. Use it only for relative comparison within the same robot and unit system.

# Conclusion
The above demonstrates how to perform symbolic analysis of a robotic system. We start with a URDF description, convert it into an ETS and then derive a symbolic expression for the pose of the robot. By selectively filtering out small coefficients we distil the expression into a more computationally tractable form. After trying (and failing) to push our analysis further using symbolic means, we switched to numerically calculating the Yoshikawa manipulability metric and identified near-singularity configurations.

# Where To Now?
If you'd like to continue on the symbolic analysis track, check out our [differential kinematics](11-differntial-kinematics.ipynb) notebook. Alternatively, check out how to analyze manipulability using a simulator in our [simulating manipulability](24-simulating-manipulability.ipynb)
